In [1]:
import json
import re
import random
from bs4 import BeautifulSoup
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


convert multiverse to singleverse

In [ ]:
with open('/content/drive/MyDrive/total_temp.jsonl', 'r',encoding="utf8") as json_file:
    json_list = list(json_file)

with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

dict_data_list = []
exception_list = []

for json_str in json_list:

    result = json.loads(json_str)
    context = result['context']
    context = re.sub("[\(\[].*?[\)\]]", "", context)
    context = re.sub(" +", " ", context)
    answer = result['answers']
    answer_freq = context.count(answer)

    new_data = {}
    new_data['question'] = result['question']
    answer_dict = {}

    if 's' in result['pq_id']:
        new_data['pq_id'] = result['pq_id']

        if '*' in context:
            markers = findOccurrences(context, '*')
            answer_dict['text'] = result['answers']
            answer_dict['start_char'] = markers[0]
            new_data['answers'] = [answer_dict]
            new_data['context'] = context.replace('*', '')
        else:
            new_data['context'] = context
            answer_dict['text'] = result['answers']
            answer_dict['start_char'] = context.find(result['answers'])
            new_data['answers'] = [answer_dict]

    elif 's' not in result['pq_id']:
        verses = (result['pq_id'].split(":", 1)[1]).split('_')[0]
        start_verse = int(verses.split('-')[0])
        end_verse = int(verses.split('-')[1])
        chapter = result['pq_id'].split(":", 1)[0]

        if start_verse == end_verse:
            new_data['pq_id'] = 's'+ str(chapter) + '.' + str(start_verse)

            if '*' in context:
                markers = findOccurrences(context, '*')
                answer_dict['text'] = result['answers']
                answer_dict['start_char'] = markers[0]
                new_data['answers'] = [answer_dict]
                new_data['context'] = context.replace('*', '')
            else:
                new_data['context'] = context
                answer_dict['text'] = result['answers']
                answer_dict['start_char'] = context.find(result['answers'])
                new_data['answers'] = [answer_dict]

        else:
            verse_list = []
            for i in range(start_verse, end_verse+1):
                verse_dict = {}
                verse_id = 's'+ str(chapter) + '.' + str(i)
                try:
                  verse_content = soup.find(id=verse_id).contents[0]
                  verse_content = re.sub("[\(\[].*?[\)\]]", "", verse_content)
                  verse_content = re.sub(" +", " ", verse_content)
                except:
                  print('verse_id: ' + str(verse_id) + 'pq_id: ' + str(result['pq_id']))

                verse_dict['id'] = verse_id
                verse_dict['content'] = verse_content
                verse_list.append(verse_dict)

            if '*' in context:
                counter = 0
                id = ''
                new_content = ''

                clean_verse = context.replace('*', '')
                for verse in verse_list:
                    if verse['content'] in context:
                        counter+=1
                    else:
                        id = verse['id']
                        new_content = verse['content']


                if counter == len(verse_list)-1 and id:
                    new_data['pq_id'] = id
                    new_data['context'] = new_content
                    if new_content.count(result['answers']) == 1:
                        start_char = new_content.find(result['answers'])
                        answer_dict['text'] = result['answers']
                        answer_dict['start_char'] = new_content.find(result['answers'])
                        new_data['answers'] = [answer_dict]
                    else:
                        for verse in verse_list:
                            if verse['id'] != id:
                                context = context.replace(verse['content'], '')

                        markers = findOccurrences(context, '*')
                        answer_dict['text'] = result['answers']
                        answer_dict['start_char'] = markers[0]
                        new_data['answers'] = [answer_dict]
                        new_data['context'] = context.replace('*', '')

                else:
                  pass
                    # print(result['pq_id'])
                    # exception_list.append(result)
                # markers = findOccurrences(context, '*')
                # passage = context.replace('*', '')

            else:
                for verse in verse_list:
                    if answer in verse['content']:
                        new_data['pq_id'] = verse['id']
                        new_data['context'] = verse['content']
                        answer_dict['text'] = result['answers']
                        answer_dict['start_char'] = verse['content'].find(result['answers'])
                        new_data['answers'] = [answer_dict]
                        break


    if 'answers' in new_data.keys():
        dict_data_list.append(new_data)
    else:
        exception_list.append(result)


In [ ]:
len(exception_list)

109

In [ ]:
len(dict_data_list)

4755

In [ ]:
import json
import random
import math

random.shuffle(dict_data_list)
split_per = math.trunc(0.8 * len(dict_data_list))

with open('/content/drive/MyDrive/train_v5.json', 'w', encoding = 'utf_8_sig') as fp:
  json.dump(dict_data_list[:split_per], fp, ensure_ascii = False)


with open('/content/drive/MyDrive/dev_v5.json', 'w', encoding = 'utf_8_sig') as fout:
  json.dump(dict_data_list[split_per:], fout, ensure_ascii = False)

save dataset with squad format

In [ ]:
new_dict_data_list = []
for data in dict_data_list:
  new_data = {}
  new_data['title'] = ''
  # try:
  new_data['paragraphs'] = [{'context': data['context'], 'qas':[{'answers':[{'answer_start':data['answers'][0]['start_char'], 'text':data['answers'][0]['text']}],'id':data['pq_id'], 'question': data['question'], 'is_impossible': False}]}]
  # except:
    # print(data)
  new_dict_data_list.append(new_data)

final_data_train = {}
final_data_dev = {}

final_data_train['data'] = new_dict_data_list[:split_per]
final_data_dev['data'] = new_dict_data_list[split_per:]

with open('/content/drive/MyDrive/train_SQuAD_format_v5.json', 'w') as fp:
  json.dump(final_data_train, fp, ensure_ascii = False)

with open('/content/drive/MyDrive/dev_SQuAD_format_v5.json', 'w') as fp:
  json.dump(final_data_dev, fp, ensure_ascii = False)

# DPR dataset creation

In [3]:
%%capture
!pip install pyserini==0.22.0
!pip install faiss-cpu

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [4]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('/content/drive/MyDrive/indexes/sample_collection_jsonl')
searcher.set_language('fa')
hits = searcher.search('لات')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')


 1 s53.19 4.84050


In [16]:
with open('/content/drive/MyDrive/train_v5.json', 'r', encoding = 'utf-8-sig') as f:
  dict_data_list = json.load(f)

with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

searcher = LuceneSearcher('/content/drive/MyDrive/indexes/sample_collection_jsonl')
searcher.set_language('fa')

dpr_data_list = []

for qa in dict_data_list:
  question = qa['question']
  dpr_data = {}
  dpr_data['dataset'] = 'Quran_QA'
  dpr_data['question'] = qa['question']
  dpr_data['answers'] = qa['answers']
  dpr_data['positive_ctxs'] = [{'title': '' , 'text': qa['context'],
                                'score': 1000, 'title_score':1, 'passage_id':qa['pq_id']}]
  dpr_data['negative_ctxs'] = []

  hits = searcher.search(question)
  hard_negatives = []
  # if len(hits)>5:
  #   negatives_number = 5
  # else:
  #   negatives_number = len(hits)

  for i in range(len(hits)):
    verse_data = {}
    # print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    verse_id = hits[i].docid
    verse_content = soup.find(id=verse_id).contents[0]
    if qa['answers'][0]['text'] not in verse_content:
      verse_data['passage_id'] = verse_id
      verse_data['text'] = verse_content
      verse_data['title'] = ''
      verse_data['score'] = hits[i].score
      verse_data['title_score'] = 0
      hard_negatives.append(verse_data)

    if len(hard_negatives) == 5:
      break

  dpr_data['hard_negative_ctxs'] = hard_negatives

  dpr_data_list.append(dpr_data)

  del dpr_data, hard_negatives, hits


In [17]:
len(dpr_data_list)

3804

In [18]:
with open('/content/drive/MyDrive/DPR_train_v2.json', 'w') as fout:
  json.dump(dpr_data_list, fout, ensure_ascii = False)

In [19]:
with open('/content/drive/MyDrive/dev_v5.json', 'r', encoding = 'utf-8-sig') as f:
  dict_data_list = json.load(f)

with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

searcher = LuceneSearcher('/content/drive/MyDrive/indexes/sample_collection_jsonl')
searcher.set_language('fa')

dpr_data_list = []

for qa in dict_data_list:
  question = qa['question']
  dpr_data = {}
  dpr_data['dataset'] = 'Quran_QA'
  dpr_data['question'] = qa['question']
  dpr_data['answers'] = qa['answers']
  dpr_data['positive_ctxs'] = [{'title': '' , 'text': qa['context'],
                                'score': 1000, 'title_score':1, 'passage_id':qa['pq_id']}]
  dpr_data['negative_ctxs'] = []

  hits = searcher.search(question)
  hard_negatives = []
  # if len(hits)>5:
  #   negatives_number = 5
  # else:
  #   negatives_number = len(hits)

  for i in range(len(hits)):
    verse_data = {}
    # print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    verse_id = hits[i].docid
    verse_content = soup.find(id=verse_id).contents[0]
    if qa['answers'][0]['text'] not in verse_content:
      verse_data['passage_id'] = verse_id
      verse_data['text'] = verse_content
      verse_data['title'] = ''
      verse_data['score'] = hits[i].score
      verse_data['title_score'] = 0
      hard_negatives.append(verse_data)

    if len(hard_negatives) == 5:
      break

  dpr_data['hard_negative_ctxs'] = hard_negatives

  dpr_data_list.append(dpr_data)

  del dpr_data, hard_negatives, hits


with open('/content/drive/MyDrive/DPR_dev_v2.json', 'w') as fout:
  json.dump(dpr_data_list, fout, ensure_ascii = False)